In [ ]:
#6.Cluster the publish time into 10-minute intervals (e.g. from 02:20 to 02:30)

In [3]:
#to solve this question first I need to have the solution for question 1, that I have get the file from my teammate.
#because the file was larger than 100MB we could not load it in the github.
import re
import pandas as pd

final_df = pd.read_csv(r'C:\Users\Utente\youtube\data\final_df.csv')
print(final_df.head())

      video_id trending_date  \
0  n1WpP7iowLc      17.14.11   
1  0dBIkQ4Mz1M      17.14.11   
2  5qpjK5DgCt4      17.14.11   
3  d380meD0W0M      17.14.11   
4  2Vv-BfVoq4g      17.14.11   

                                               title channel_title  \
0         Eminem - Walk On Water (Audio) ft. Beyoncé    EminemVEVO   
1                      PLUSH - Bad Unboxing Fan Mail     iDubbbzTV   
2  Racist Superman | Rudy Mancuso, King Bach & Le...  Rudy Mancuso   
3                           I Dare You: GOING BALD!?      nigahiga   
4        Ed Sheeran - Perfect (Official Music Video)    Ed Sheeran   

   category_id              publish_time  \
0           10  2017-11-10T17:00:03.000Z   
1           23  2017-11-13T17:00:00.000Z   
2           23  2017-11-12T19:05:24.000Z   
3           24  2017-11-12T18:01:41.000Z   
4           10  2017-11-09T11:04:14.000Z   

                                                tags     views    likes  \
0  Eminem|"Walk"|"On"|"Water"|"Aftermath/Shady

In [8]:
# 1. Convert the column to datetime objects
final_df['publish_time'] = pd.to_datetime(final_df['publish_time'])
#REMOVE the timezone (I checked it in the file, it was the same in all countries)
final_df['publish_time'] = final_df['publish_time'].dt.tz_localize(None)

# 2. Round down to the nearest 10 minutes
# Example: 02:23:00 becomes 02:20:00
interval_start = final_df['publish_time'].dt.floor('10min')

# 3. Create your "Start-End" string format
# Adds 10 minutes to the start time to get the end time
final_df['interval_10min'] = (
    interval_start.dt.strftime('%H:%M') + "-" + 
    (interval_start + pd.Timedelta(minutes=10)).dt.strftime('%H:%M')
)

print(final_df[['publish_time', 'interval_10min']].head())

         publish_time interval_10min
0 2017-11-10 17:00:03    17:00-17:10
1 2017-11-13 17:00:00    17:00-17:10
2 2017-11-12 19:05:24    19:00-19:10
3 2017-11-12 18:01:41    18:00-18:10
4 2017-11-09 11:04:14    11:00-11:10


In [ ]:
#7.For each interval, determine the number of videos, average number of likes and of dislikes.

In [9]:
interval_stats = final_df.groupby('interval_10min').agg(
    num_videos=('video_id', 'count'),
    avg_likes=('likes', 'mean'),
    avg_dislikes=('dislikes', 'mean')
).reset_index() #to reset 'interval_10min' from being index (was converted by groupby) into a flat table
print(interval_stats.head())

  interval_10min  num_videos     avg_likes  avg_dislikes
0    00:00-00:10        2913  60951.483350   3787.232750
1    00:10-00:20        1522  22553.870565   1437.457293
2    00:20-00:30        1248  21258.370192   1066.330128
3    00:30-00:40        1625  36604.352000    949.439385
4    00:40-00:50        1283  41770.614186   1889.012471


In [ ]:
#8.For each tag, determine the number of videos. Notice that tags contains a string with several tags.

In [53]:
from collections import Counter


all_tags = []

for tags_string in final_df['tags']:
    if pd.notna(tags_string):  # Check if not NaN

        # Lowercase so Funny = funny
        clean_string = str(tags_string).lower()

        # Split on ANY non-letter/number character. I detected any sepraters such as |, /, #, *... so I count all of them as sepraters
        tags_list = re.split(r'[^a-z0-9]+', clean_string)

        # Clean tags
        tags_set = set()   # ensures one count per video
        for tag in tags_list:
            tag = tag.strip()
            if tag and tag != '[none]' and tag != "":
                tags_set.add(tag)

        all_tags.extend(tags_set) #extend: add tags one by one

# Count occurrences of each tag
tag_counts = Counter(all_tags)
#converting to a dataframe to print it
tag_stats = pd.DataFrame(tag_counts.items(), columns=['tag', 'repeat_nums'])
print("\n first 30 tags with the numbers of repeats:\n")
print(tag_stats.head(30))


 first 30 tags with the numbers of repeats:

           tag  repeat_nums
0       eminem          670
1        shady          217
2    aftermath          421
3           on        10502
4        water         1940
5          rap         6935
6   interscope          985
7         walk          830
8         best        13086
9     unboxing         1940
10   idubbbztv           42
11    packages          131
12        mail          165
13         bad         3169
14      things         2653
15        chop          189
16  idubbbztv2           33
17       plush           24
18    chontent            9
19         fan         1020
20    plushies           22
21     license           52
22        love         8336
23      sarkis          583
24     getting          568
25      racist          296
26      inanna          591
27   pineapple          238
28        rudy          446
29      alesso          714


In [ ]:
#9. Find the tags with the largest number of videos

In [54]:
tag_stats = tag_stats.sort_values('repeat_nums', ascending=False)
print(tag_stats.head(20))

         tag  repeat_nums
2246    2018        43586
134      the        43375
804     none        37705
50     video        31508
432       tv        30323
192      new        27643
62     funny        24322
982       de        23794
94      news        23710
72    comedy        22029
866        a        21996
49     music        21333
45         s        21116
284     show        20994
299       of        19061
162     2017        18421
172       to        18035
175       in        17357
198   latest        16258
537   videos        15513


In [ ]:
#10. For each (tag, country) pair, compute average ratio likes/dislikes

In [59]:
import numpy as np

df_tags = final_df[['country', 'tags', 'likes', 'dislikes']].copy()

# Clean & split tags
df_tags['tags'] = (
    df_tags['tags']
    .astype(str)
    .str.lower()
    .str.split(r'[^a-z0-9]+', regex=True)
)

# Explode
df_exploded = df_tags.explode('tags')

# Remove empty, [none], nan tags
df_exploded = df_exploded[
    (df_exploded['tags'] != "") &
    (df_exploded['tags'] != "[none]") &
    (df_exploded['tags'] != "nan") &
    (df_exploded['tags'].str.contains('[a-z0-9]', regex=True))
]


# Avoid division by zero → replace dislikes = 0 with 1 to calculate number of likes for those videos
df_exploded['dislikes'] = df_exploded['dislikes'].replace(0, 1)
df_exploded['ratio'] = df_exploded['likes'] / df_exploded['dislikes']

# Group
tag_country_stats = (
    df_exploded.groupby(['tags', 'country'])['ratio']
    .mean()
    .reset_index()
    .rename(columns={'tags': 'tag', 'ratio': 'avg_like/dislike_ratio'})
)

print(tag_country_stats.tail(30))

                         tag         country  avg_like/dislike_ratio
317478              zydrunas         Germany               20.750000
317479      zydrunassavickas         Germany              392.000000
317480               zygarde          Canada              277.584615
317481                zygote          Canada                7.550407
317482                zygote         Germany                7.456664
317483                zygote   United States                7.521840
317484                zyklon         Germany               36.253186
317485      zyklonabscheider         Germany               19.945946
317486           zylberstein          France               17.000000
317487              zylinder         Germany              159.251956
317488          zylinderkopf         Germany               67.899178
317489  zylinderkopfdichtung         Germany               60.764338
317490      zylinderkopfriss         Germany               89.696023
317491                 zylka   Uni